In [56]:
import json
import numpy as np
# Read from JSON file
num_qubit = 10  # Length of each vector
with open("LABS/"+"labs_paulis/labs_n"+ str(num_qubit) +"_layers1.json", "r") as file:
    paulis = json.load(file)

In [57]:
binary_vectors = []
for pauli in paulis:
    vector = [False]*len(pauli)
    for i, bit in enumerate(pauli):
        if bit=='Z':
            vector[i]=True
    if sum(vector)!=0:
        binary_vectors.append(vector)

In [58]:
import json

with open("LABS/"+ str(num_qubit) +"_"+ str(1) +".json", "w") as f:
    json.dump(binary_vectors, f)

### Qiskit transpile

In [59]:
import json
import numpy as np
# Read from JSON file
num_qubit = 20  # Length of each vector
with open("LABS/"+ str(num_qubit) +"_"+ str(1) +".json", "r") as file:
    boolean_functions = json.load(file)

F = []
one_term_boolean_functions = []
for f in boolean_functions:
    if sum(f)==0:
        print("Incorrect! Containing all False boolean function")
        break
    if sum(f)==1:
        one_term_boolean_functions.append(f)
    if sum(f)>1:
        F.append(f)

In [60]:
import numpy as np

def hamming_distance(vec1, vec2):
    """
    Computes the Hamming distance between two binary vectors.
    
    :param vec1: First binary vector (list of 0/1 or True/False)
    :param vec2: Second binary vector (list of 0/1 or True/False)
    :return: Integer Hamming distance
    """
    return sum(a != b for a, b in zip(vec1, vec2))

def select_best_vector(reference_list, candidate_list):
    """
    Selects the vector from candidate_list that has the minimum total Hamming weight 
    with respect to all vectors in reference_list.
    
    :param reference_list: List of binary vectors (comparison base)
    :param candidate_list: List of binary vectors (selection pool)
    :return: The best binary vector from candidate_list
    """
    best_vector = None
    min_hamming_sum = float('inf')

    for candidate in candidate_list:
        total_hamming = sum(hamming_distance(candidate, ref) for ref in reference_list)
        
        if total_hamming < min_hamming_sum:
            min_hamming_sum = total_hamming
            best_vector = candidate

    return best_vector

ordered_F = []
candidate_F = [f for f in F]
while candidate_F:
    # Select the best vector
    best_vector = select_best_vector(ordered_F, candidate_F)
    ordered_F.append(best_vector)
    candidate_F.remove(best_vector)

In [6]:
import os
import sys

here = os.path.dirname('Benchmark.ipynb')

sys.path.append(os.path.join(here, '..'))

In [62]:
from QAOA_benchmark import boolean_func_to_normal_qaoa_circuit
import time
start_time = time.time()
org_qc, theta_2 = boolean_func_to_normal_qaoa_circuit(ordered_F,1)

parameter_values = [np.pi/5, np.pi/4]
parameter_dict = {param: value for param, value in zip(theta_2, parameter_values)}

# Bind the parameters
bound_org_qc = org_qc.assign_parameters(parameter_dict)

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
pm = generate_preset_pass_manager(optimization_level=3)
candidate_circuit = pm.run(bound_org_qc)

end_time = time.time()
elapsed_time = end_time - start_time
print(elapsed_time)

0.4547579288482666


In [63]:
candidate_circuit.count_ops()

OrderedDict([('cx', 3192), ('rz', 615), ('h', 20), ('rx', 20)])

In [64]:
candidate_circuit.depth()

3154

In [65]:
from qiskit import QuantumCircuit, qpy
path = '/Users/xinpengli/Desktop/QAOAZ3/'
with open(path+"Experiment/LABS/logical/"+ str(num_qubit) +"_"+ str(1) +"_logical_circuit.qpy", "wb") as qpy_file_write:
    qpy.dump(candidate_circuit, qpy_file_write)

In [51]:
from qiskit_util import  remove_unused_qubit, Fake127_transpile
import time
start_time = time.time()
transpiled_bound_org_qc,subcoupling_map = Fake127_transpile(candidate_circuit, basis_gate=False)
end_time = time.time()
elapsed_time = end_time - start_time
print(elapsed_time)

operator count:  OrderedDict([('cx', 3192), ('swap', 3131), ('rz', 615), ('h', 20), ('rx', 20)])
depth:  4729
0.728644847869873


In [52]:
# from qiskit_util import swap_to_cnot
def swap_to_cnot(qc):
    new_qc = QuantumCircuit(qc.qubits, qc.clbits)
    for gate in qc.data:
        if gate.name == 'swap':
            new_qc.cx(gate.qubits[0], gate.qubits[1])
            new_qc.cx(gate.qubits[1], gate.qubits[0])
            new_qc.cx(gate.qubits[0], gate.qubits[1])
        else:
            new_qc.append(gate)
    return new_qc
no_swap_recovered_transpiled_bound_org_qc  = swap_to_cnot(transpiled_bound_org_qc)

In [53]:
no_swap_recovered_transpiled_bound_org_qc.depth()

7959

In [54]:
no_swap_recovered_transpiled_bound_org_qc.count_ops()

OrderedDict([('cx', 12585), ('rz', 615), ('h', 20), ('rx', 20)])

In [55]:
from qiskit import QuantumCircuit, qpy
import json
with open(path+"Experiment/LABS/qiskit/"+ str(num_qubit)+"_qiskit_circuit.qpy", "wb") as qpy_file_write:
    qpy.dump(transpiled_bound_org_qc, qpy_file_write)

with open(path+"Benchmark/LABS_layout/"+ str(num_qubit) +"_IBM_127.json", "w") as f:
    json.dump(subcoupling_map, f)